In [6]:
from google.colab import drive
import os

drive.mount('/content/drive')

DATASET_PATH = '/content/drive/MyDrive/cvpr_final'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from PIL import Image

deleted_count = 0
for subdir, dirs, files in os.walk(DATASET_PATH):
    for file in files:
        filepath = os.path.join(subdir, file)
        try:
            img = Image.open(filepath)
            img.verify()
        except:
            os.remove(filepath)
            deleted_count += 1

print(f"Cleaned up {deleted_count} bad files from local storage.")

Cleaned up 208 bad files from local storage.


In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (150, 150)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 1493 images belonging to 97 classes.
Found 354 images belonging to 97 classes.


In [12]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50
)

model.save('student_recognition_model.h5')
print("Training done. Model saved as 'student_recognition_model.h5'")

Epoch 1/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 116s 2s/step - accuracy: 0.0238 - loss: 4.5533 - val_accuracy: 0.0565 - val_loss: 4.4078
Epoch 2/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.0468 - loss: 4.4358 - val_accuracy: 0.0819 - val_loss: 4.3377
Epoch 3/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.0538 - loss: 4.3736 - val_accuracy: 0.0650 - val_loss: 4.2086
Epoch 4/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.0695 - loss: 4.2118 - val_accuracy: 0.0847 - val_loss: 3.9340
Epoch 5/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.1098 - loss: 4.0083 - val_accuracy: 0.1497 - val_loss: 3.6450
Epoch 6/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.1111 - loss: 3.7931 - val_accuracy: 0.1638 - val_loss: 3.4341
Epoch 7/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.1497 - loss: 3.5497 - val_accuracy: 0.2260 - val_loss: 3.1908
Epoch 8/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.1755 - loss: 3.4355 - val_accuracy: 0.2542 - v

Training done. Model saved as 'student_recognition_model.h5'


In [13]:
labels = list(train_generator.class_indices.keys())
with open("labels.txt", "w") as f:
    f.write("\n".join(labels))